# **Cargar los datos**

In [1]:
pip install biosignalsnotebooks

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.1/39.1 MB 29.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 105.6 MB/s eta 0:00:00
  Created wheel for libmagic: filename=libmagic-1.0-py3-none-any.whl size=4269 sha256=8d493b8eacb911b51d5aab336832c549af3ed4607f98634a016ca687d2685f5e
  Stored in directory: /root/.cache/pip/wheels/31/ce/d8/099fbd4dbfa5da5596a39eee8a17d2e1bcc29006c07d3563b7
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=5bbf1bbe984b5eda2d97f040376809c9efb0e124d69fa99da58c4bceebf311ca
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d9758

In [2]:

#package usado para cargar datos desde un archivo h5
from h5py import File

#package para trabajar con arreglos
from numpy import array
from numpy import cumsum, concatenate, zeros, linspace, average, power, absolute, mean, std, max, array, diff, where

import numpy as np
#biosignalnotebooks python package
import biosignalsnotebooks as bsnb
from scipy.signal import butter, lfilter
from scipy.stats import linregress

from bokeh.plotting import figure, show

#n
import math
from scipy.integrate import cumtrapz
from scipy.signal import welch

In [3]:
#f = open("EMGTayel.txt","r")
#raw_data = f.readline()  # con f.read() leemos todo el contenido
#f.close()

#raw_data
File_name="emg.h5"
h5_object=File(File_name)


In [4]:
data, header=bsnb.load("emg.h5",get_header=True)

In [5]:
channel = list(data.keys())[0]

In [6]:
# Sampling rate and acquired data
sr = header["sampling rate"]
device = header["device"]

# Signal Samples
signal = data[channel]
time = bsnb.generate_time(signal)

# **Mostrar los datos**

In [7]:
signal= data["CH1"]
time= bsnb.generate_time(signal, header["sampling rate"])

bokeh_figure= figure(x_axis_label ='Time (s)', y_axis_label= 'Raw Data')
bokeh_figure.line(time,signal,legend_label="Datos Originales")
show(bokeh_figure)

# **Detección de eventos**

In [8]:
pre_pro_signal=signal-average(signal)

#filtrado

#sr= 0.5
low_cutoff= 10
high_cutoff= 300
pre_pro_signal=bsnb.aux_functions._butter_bandpass_filter(pre_pro_signal,low_cutoff,high_cutoff,sr)

In [9]:
#umbral
avg_pre_pro_signal= np.mean(pre_pro_signal)
std_pre_pro_signal= std(pre_pro_signal)

In [10]:
# Regression function.
def normReg(thresholdLevel):
    threshold_0_perc_level = (- avg_pre_pro_signal) / float(std_pre_pro_signal)
    threshold_100_perc_level = (max(smooth_signal) - avg_pre_pro_signal) / float(std_pre_pro_signal)
    m, b = linregress([0, 100], [threshold_0_perc_level, threshold_100_perc_level])[:2]
    return m * thresholdLevel + b 
    

In [11]:
# [Application of TKEO Operator]
tkeo = []
for i in range(0, len(pre_pro_signal)):
    if i == 0 or i == len(pre_pro_signal) - 1:
        tkeo.append(pre_pro_signal[i])
    else:
        tkeo.append(power(pre_pro_signal[i], 2) - (pre_pro_signal[i + 1] * pre_pro_signal[i - 1]))


In [12]:
# [Signal Rectification]
rect_signal = absolute(tkeo)

In [13]:
# [First Moving Average Filter]
rect_signal = bsnb.aux_functions._moving_average(rect_signal, sr / 10)

In [14]:
# Smoothing level [Size of sliding window used during the moving average process (a function of sampling frequency)]
smoothing_level_perc = 20 # Percentage.
smoothing_level = int((smoothing_level_perc / 100) * sr)

In [15]:
# [Second Smoothing Phase]
smooth_signal = []
for i in range(0, len(rect_signal)):
    if smoothing_level < i < len(rect_signal) - smoothing_level:
        smooth_signal.append(mean(rect_signal[i - smoothing_level:i + smoothing_level]))
    else:
        smooth_signal.append(0)

In [16]:
#umbrales externos
threshold_level=10
threshold_level_norm_10=normReg(threshold_level)

threshold_level=80
threshold_level_norm_80=normReg(threshold_level)

threshold_10=avg_pre_pro_signal+threshold_level_norm_10*std_pre_pro_signal
threshold_l80=avg_pre_pro_signal+threshold_level_norm_80*std_pre_pro_signal



In [17]:
#onda cuadrada segun los periodos de activacion
binary_signal=[]
for i in range(0,len(time)):
  if smooth_signal[i] >= threshold_10:
    binary_signal.append(1)
  else:
    binary_signal.append(0)


# **Extracción de características**

In [18]:
resolution=1

In [19]:
fs=header["sampling rate"]
signal= bsnb.raw_to_phy("EMG", device, data[channel], resolution, option ="mV")
time= linspace(0,len(signal)/ fs, len(signal))

burst_begin, burst_end = bsnb.detect_emg_activations(signal, fs, smooth_level=20, threshold_level=10, time_units=True,plot_result=True)[:2]

max_sample_value= max(signal)
min_sample_value= min(signal)

avg_sample_value= average(signal)
std_sample_value= std(signal)

time_param_dict = { "Maximum EMG": max_sample_value, "Minimum EMG": min_sample_value, "Average EMG": avg_sample_value, "Standard Deviation EMG": std_sample_value}

rms= math.sqrt(sum(signal*signal)/ len(signal))

area= cumtrapz(signal)

f,P= welch(signal, fs=fs, window= 'hann', noverlap=0, nfft= int(256.))

area_freq= cumtrapz(P,f,initial=0)
total_power= area_freq[-1]
median_freq= f[where(area_freq >= total_power /2)[0][0]]
f_max= f[np.argmax(P)]